<a href="https://colab.research.google.com/github/ATIK2110018/geospatial-python/blob/main/teesta_project_lulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install geemap -q

In [8]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='atik-2110018')
Map = geemap.Map()

In [9]:
shapefile_path = '/content/drive/MyDrive/GIS data/teesta_project/aoi/aoi.shp'

In [10]:
!pip install pycrs -q

In [11]:
aoi = geemap.shp_to_ee(shapefile_path)

In [12]:
image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(aoi)
    .filterDate("2024-12-01", "2025-01-20")
    .filter(ee.Filter.lt('CLOUD_COVER', 1))
    .sort("CLOUD_COVER")
    .mosaic()
    .select('SR_B[1-7]')
)

vis_params = {"min": 0, "max": 25000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

Map.centerObject(aoi, 8)
Map.addLayer(image, vis_params, "Landsat-9")
Map.addLayer(aoi, {}, "AOI")

In [13]:
land_2024= image.clip(aoi)
Map.addLayer(land_2024, vis_params, "land_2024")
Map

Map(center=[26.054918001644495, 89.11593911157134], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
# prompt: export land_2024

geemap.ee_export_image_to_drive(
    land_2024,
    description='land_2024',
    folder='earthengine_export',
    region=aoi.geometry(),
    scale=30
)

In [15]:
!pip install rasterio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 36.1 MB/s eta 0:00:00


In [16]:
props = image.propertyNames()

print(image.get("CLOUD_COVER").getInfo())

None


In [17]:
training = land_2024.sample(
    region=aoi.geometry(), # Provide the geometry of the aoi as the region
    scale=30,
    numPixels=10000,
    seed=0,
    geometries=True
)

Map.addLayer(training, {}, "training", False)
Map

Map(center=[26.054918001644495, 89.11593911157134], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
n_clusters = 3
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [19]:
result = land_2024.cluster(clusterer)

Map.addLayer(result.randomVisualizer(), {}, "clusters")

In [20]:
class_names = ['Bar', 'Vegetation', 'Water']
class_palette = {i: name for i, name in enumerate(class_names)}

color_dict = {
    'blue': (0, 0, 255),
    'green': (0, 128, 0),
    'red': (255, 0, 0)
}

class_colors = ['red', 'green', 'blue']

legend_dict = {class_names[i]: color_dict[class_colors[i]] for i in range(len(class_names))}

classified_image = result.remap(
    ee.List(list(class_palette.keys())),
    ee.List(list(class_palette.keys())),
    0
).rename('land_cover')

Map.addLayer(
    classified_image,
    {'min': 0, 'max': 2, 'palette': class_colors},
    'Classified Image'
)

Map.add_legend(legend_dict=legend_dict, title="Land Cover Classification")

Map

Map(center=[26.054918001644495, 89.11593911157134], controls=(WidgetControl(options=['position', 'transparent_…

In [21]:
original_values = [0, 1, 2]
target_values = [2, 4, 6]

classified_image_remapped = classified_image.remap(
    ee.List(original_values),
    ee.List(target_values),
    0
).rename('land_cover')

In [22]:
geemap.ee_export_image(
    classified_image_remapped,
    filename='classified_image.tif',
    scale=30,
    region=aoi.geometry(),
    file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to /content/classified_image.tif


In [23]:
import ee
from geemap import ee_export_image_to_drive

ee_export_image_to_drive(
    image=land_2024,
    description='land_2024_drive_export',
    folder='GEE',
    fileNamePrefix='land_2024',
    region=aoi.geometry(),
    scale=30
)